# 🚀 Tối Ưu Xử Lý Dữ Liệu - Giai Đoạn 1

## Mục tiêu
Triển khai **Giai đoạn 1** của kế hoạch tối ưu mô hình QCAAPatchTF:
- 📊 **Chuẩn hóa Target**: StandardScaler cho `order_count` 
- 🔄 **Mã hóa chu kỳ**: Sin/Cos cho đặc trưng thời gian
- 🏢 **Đặc trưng thị trường**: Tỷ lệ và thị phần các market
- 📅 **Lịch kinh doanh**: Cuối tháng, cuối quý indicators
- 📈 **Market dynamics**: Trend 7 ngày và volatility

**Kết quả dự kiến**: MSE: 20,437 → 10,000-12,000 (cải thiện 50-60%)

## 1. 📦 Environment Setup and Library Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import pickle
import os
from datetime import datetime
import joblib

# Cấu hình
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

# Set random seed cho reproducibility
np.random.seed(42)

print("✅ Đã import tất cả thư viện cần thiết!")
print(f"📅 Thời gian bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Đã import tất cả thư viện cần thiết!
📅 Thời gian bắt đầu: 2025-08-22 00:05:57


## 2. 📂 Load and Explore Original Dataset

In [2]:
# Load dữ liệu gốc
data_path = '../dataset/supply_chain_processed.csv'
df_original = pd.read_csv(data_path)

print("📊 Thông tin dataset gốc:")
print(f"- Kích thước: {df_original.shape}")
print(f"- Cột: {df_original.columns.tolist()}")
print(f"- Kiểu dữ liệu:")
print(df_original.dtypes)

# Hiển thị vài dòng đầu
print("\n🔍 5 dòng đầu:")
df_original.head()

📊 Thông tin dataset gốc:
- Kích thước: (765, 23)
- Cột: ['order_date_only', 'Market', 'Days for shipping (real)', 'Late_delivery_risk', 'Order Item Product Price', 'Order Item Discount Rate', 'Order Item Profit Ratio', 'Order Profit Per Order', 'Order Item Quantity', 'Sales', 'Order Item Total', 'customer_segment_consumer_pct', 'customer_segment_corporate_pct', 'customer_segment_home_office_pct', 'category_diversity_index', 'price_volatility', 'order_count', 'day_of_week', 'day_of_month', 'month', 'is_weekend', 'days_since_start', 'Market_encoded']
- Kiểu dữ liệu:
order_date_only                      object
Market                               object
Days for shipping (real)            float64
Late_delivery_risk                  float64
Order Item Product Price            float64
Order Item Discount Rate            float64
Order Item Profit Ratio             float64
Order Profit Per Order              float64
Order Item Quantity                   int64
Sales                            

,order_date_only,Market,Days for shipping (real),Late_delivery_risk,Order Item Product Price,Order Item Discount Rate,Order Item Profit Ratio,Order Profit Per Order,Order Item Quantity,Sales,...,customer_segment_home_office_pct,category_diversity_index,price_volatility,order_count,day_of_week,day_of_month,month,is_weekend,days_since_start,Market_encoded
0,2017-05-22,Europe,3.727273,0.611570,128.070003,0.104545,0.142893,24.441735,255,22606.920411,...,0.123967,0.887508,0.1,182,0,22,5,0,0,0
1,2017-05-22,LATAM,3.396341,0.469512,136.390735,0.111402,0.104695,18.309634,351,32234.440596,...,0.207317,0.894036,0.1,164,0,22,5,0,0,1
2,2017-05-22,USCA,3.032787,0.480874,124.605522,0.098306,0.144262,21.100055,410,35530.630638,...,0.174863,0.892950,0.1,183,0,22,5,0,0,2
3,2017-05-23,Europe,3.739130,0.543478,131.515764,0.092989,0.147609,24.574185,412,37639.590661,...,0.130435,0.897271,0.1,184,1,23,5,0,1,0
4,2017-05-23,LATAM,3.561728,0.580247,124.624818,0.093210,0.137716,25.668457,377,33127.360613,...,0.154321,0.890794,0.1,162,1,23,5,0,1,1


In [3]:
# Kiểm tra missing values
print("❌ Missing values:")
missing_counts = df_original.isnull().sum()
missing_counts = missing_counts[missing_counts > 0]
if len(missing_counts) > 0:
    print(missing_counts)
else:
    print("✅ Không có missing values!")

# Phân tích target variable (order_count)
print(f"\n📈 Phân tích target variable 'order_count':")
print(f"- Min: {df_original['order_count'].min()}")
print(f"- Max: {df_original['order_count'].max()}")
print(f"- Mean: {df_original['order_count'].mean():.2f}")
print(f"- Std: {df_original['order_count'].std():.2f}")
print(f"- Range: {df_original['order_count'].max() - df_original['order_count'].min()}")

# Phân tích theo Market
print(f"\n🌍 Phân tích theo Market:")
market_stats = df_original.groupby('Market')['order_count'].agg(['count', 'mean', 'std', 'min', 'max'])
print(market_stats)

❌ Missing values:
✅ Không có missing values!

📈 Phân tích target variable 'order_count':
- Min: 133
- Max: 212
- Mean: 170.93
- Std: 14.05
- Range: 79

🌍 Phân tích theo Market:
        count        mean        std  min  max
Market                                        
Europe    255  170.600000  13.087110  135  203
LATAM     255  171.976471  13.914228  134  209
USCA      255  170.200000  15.083077  133  212


## 3. 📊 Target Normalization Implementation
**Mục tiêu**: Chuẩn hóa `order_count` để giảm MSE từ 20,437 → 10,000-12,000

In [4]:
# Tạo copy để xử lý
df = df_original.copy()

# Chuẩn hóa target với StandardScaler (khuyến nghị từ roadmap)
print("🎯 Áp dụng StandardScaler cho order_count...")

# Khởi tạo scaler
target_scaler = StandardScaler()

# Backup giá trị gốc
df['order_count_original'] = df['order_count'].copy()

# Fit và transform
df['order_count_normalized'] = target_scaler.fit_transform(df[['order_count']]).flatten()

print("✅ Hoàn thành chuẩn hóa target!")

# So sánh trước và sau
print("\n📊 So sánh TRƯỚC và SAU chuẩn hóa:")
print("TRƯỚC (Original):")
print(f"- Mean: {df['order_count_original'].mean():.2f}")
print(f"- Std: {df['order_count_original'].std():.2f}")
print(f"- Min: {df['order_count_original'].min()}")
print(f"- Max: {df['order_count_original'].max()}")

print("\nSAU (Normalized):")
print(f"- Mean: {df['order_count_normalized'].mean():.6f}")
print(f"- Std: {df['order_count_normalized'].std():.6f}")
print(f"- Min: {df['order_count_normalized'].min():.3f}")
print(f"- Max: {df['order_count_normalized'].max():.3f}")

# Thay thế order_count bằng version đã normalize
df['order_count'] = df['order_count_normalized']
df.drop('order_count_normalized', axis=1, inplace=True)

🎯 Áp dụng StandardScaler cho order_count...
✅ Hoàn thành chuẩn hóa target!

📊 So sánh TRƯỚC và SAU chuẩn hóa:
TRƯỚC (Original):
- Mean: 170.93
- Std: 14.05
- Min: 133
- Max: 212

SAU (Normalized):
- Mean: -0.000000
- Std: 1.000654
- Min: -2.700
- Max: 2.924


## 4. 🔄 Cyclical Feature Engineering
**Mục tiêu**: Mã hóa chu kỳ cho month, day_of_week, day_of_month với sin/cos

In [5]:
def create_cyclical_features(df, col_name, period):
    """Tạo đặc trưng chu kỳ với sin/cos"""
    angle = 2 * np.pi * df[col_name] / period
    df[f'{col_name}_sin'] = np.sin(angle)
    df[f'{col_name}_cos'] = np.cos(angle)
    return df

print("🔄 Tạo cyclical features...")

# 1. Month cyclical (12 months)
df = create_cyclical_features(df, 'month', 12)
print(f"✅ Month: sin/cos từ {df['month'].min()} đến {df['month'].max()}")

# 2. Day of week cyclical (7 days) 
df = create_cyclical_features(df, 'day_of_week', 7)
print(f"✅ Day of week: sin/cos từ {df['day_of_week'].min()} đến {df['day_of_week'].max()}")

# 3. Day of month cyclical (31 days)
df = create_cyclical_features(df, 'day_of_month', 31)
print(f"✅ Day of month: sin/cos từ {df['day_of_month'].min()} đến {df['day_of_month'].max()}")

print(f"\n📊 Đã thêm 6 features mới (3 × sin/cos)")
print(f"- month_sin, month_cos")
print(f"- day_of_week_sin, day_of_week_cos") 
print(f"- day_of_month_sin, day_of_month_cos")

# Kiểm tra ví dụ cyclical encoding
print(f"\n🔍 Ví dụ cyclical encoding cho tháng:")
example_months = df[['month', 'month_sin', 'month_cos']].drop_duplicates().sort_values('month')
print(example_months.head(10))

🔄 Tạo cyclical features...
✅ Month: sin/cos từ 1 đến 12
✅ Day of week: sin/cos từ 0 đến 6
✅ Day of month: sin/cos từ 1 đến 31

📊 Đã thêm 6 features mới (3 × sin/cos)
- month_sin, month_cos
- day_of_week_sin, day_of_week_cos
- day_of_month_sin, day_of_month_cos

🔍 Ví dụ cyclical encoding cho tháng:
     month     month_sin     month_cos
672      1  5.000000e-01  8.660254e-01
0        5  5.000000e-01 -8.660254e-01
30       6  1.224647e-16 -1.000000e+00
120      7 -5.000000e-01 -8.660254e-01
213      8 -8.660254e-01 -5.000000e-01
306      9 -1.000000e+00 -1.836970e-16
396     10 -8.660254e-01  5.000000e-01
489     11 -5.000000e-01  8.660254e-01
579     12 -2.449294e-16  1.000000e+00


## 5. 📅 Business Calendar Features
**Mục tiêu**: Tạo is_month_end, is_quarter_end, week_of_year cho supply chain planning

In [6]:
# Chuyển order_date_only về datetime nếu chưa
df['order_date_only'] = pd.to_datetime(df['order_date_only'])

print("📅 Tạo business calendar features...")

# 1. Is Month End (ngày 28-31 của tháng)
df['is_month_end'] = (df['day_of_month'] >= 28).astype(int)

# 2. Is Quarter End (tháng 3, 6, 9, 12 và cuối tháng)
quarter_end_months = [3, 6, 9, 12]
df['is_quarter_end'] = ((df['month'].isin(quarter_end_months)) & 
                        (df['day_of_month'] >= 28)).astype(int)

# 3. Week of Year
df['week_of_year'] = df['order_date_only'].dt.isocalendar().week

# 4. Week of Year cyclical (52 weeks)
df = create_cyclical_features(df, 'week_of_year', 52)

print("✅ Business calendar features:")
print(f"- is_month_end: {df['is_month_end'].sum()} ngày cuối tháng")
print(f"- is_quarter_end: {df['is_quarter_end'].sum()} ngày cuối quý")
print(f"- week_of_year: từ tuần {df['week_of_year'].min()} đến {df['week_of_year'].max()}")
print(f"- week_of_year_sin/cos: cyclical encoding")

# Kiểm tra ví dụ
print(f"\n🔍 Ví dụ ngày cuối tháng:")
month_end_examples = df[df['is_month_end'] == 1][['order_date_only', 'day_of_month', 'is_month_end']].head()
print(month_end_examples)

print(f"\n🔍 Ví dụ ngày cuối quý:")
quarter_end_examples = df[df['is_quarter_end'] == 1][['order_date_only', 'month', 'day_of_month', 'is_quarter_end']].head()
print(quarter_end_examples)

📅 Tạo business calendar features...
✅ Business calendar features:
- is_month_end: 99 ngày cuối tháng
- is_quarter_end: 30 ngày cuối quý
- week_of_year: từ tuần 1 đến 52
- week_of_year_sin/cos: cyclical encoding

🔍 Ví dụ ngày cuối tháng:
   order_date_only  day_of_month  is_month_end
18      2017-05-28            28             1
19      2017-05-28            28             1
20      2017-05-28            28             1
21      2017-05-29            29             1
22      2017-05-29            29             1

🔍 Ví dụ ngày cuối quý:
    order_date_only  month  day_of_month  is_quarter_end
111      2017-06-28      6            28               1
112      2017-06-28      6            28               1
113      2017-06-28      6            28               1
114      2017-06-29      6            29               1
115      2017-06-29      6            29               1


## 6. 🏢 Market Relationship Features
**Mục tiêu**: Tạo cross-market ratios và market share calculations

In [7]:
print("🏢 Tạo Market Relationship Features...")

# Đầu tiên cần pivot data để có order_count cho từng market theo ngày
print("📊 Pivot data theo market...")

# Tạo pivot table với order_count_original cho từng market theo ngày
market_pivot = df.pivot_table(
    index='order_date_only', 
    columns='Market', 
    values='order_count_original',  # Dùng giá trị gốc để tính ratio
    aggfunc='first'
).reset_index()

market_pivot.columns.name = None
print(f"✅ Pivot table: {market_pivot.shape}")
print(f"Markets: {[col for col in market_pivot.columns if col != 'order_date_only']}")

# Rename columns để dễ làm việc
market_pivot = market_pivot.rename(columns={
    'Europe': 'europe_orders',
    'LATAM': 'latam_orders', 
    'USCA': 'usca_orders'
})

print(market_pivot.head())

🏢 Tạo Market Relationship Features...
📊 Pivot data theo market...
✅ Pivot table: (255, 4)
Markets: ['Europe', 'LATAM', 'USCA']
  order_date_only  europe_orders  latam_orders  usca_orders
0      2017-05-22            182           164          183
1      2017-05-23            184           162          136
2      2017-05-24            182           171          163
3      2017-05-25            160           161          173
4      2017-05-26            175           190          167


In [8]:
# Tính cross-market ratios (theo roadmap)
print("🔄 Tính cross-market ratios...")

# 1. Europe vs LATAM ratio
market_pivot['europe_vs_latam_ratio'] = market_pivot['europe_orders'] / (market_pivot['latam_orders'] + 1e-6)

# 2. USCA vs Europe ratio  
market_pivot['usca_vs_europe_ratio'] = market_pivot['usca_orders'] / (market_pivot['europe_orders'] + 1e-6)

# 3. Total orders và market shares
market_pivot['total_orders'] = (market_pivot['europe_orders'] + 
                               market_pivot['latam_orders'] + 
                               market_pivot['usca_orders'])

market_pivot['europe_market_share'] = market_pivot['europe_orders'] / market_pivot['total_orders']
market_pivot['latam_market_share'] = market_pivot['latam_orders'] / market_pivot['total_orders']
market_pivot['usca_market_share'] = market_pivot['usca_orders'] / market_pivot['total_orders']

print("✅ Market relationship features:")
print(f"- europe_vs_latam_ratio: {market_pivot['europe_vs_latam_ratio'].mean():.3f} ± {market_pivot['europe_vs_latam_ratio'].std():.3f}")
print(f"- usca_vs_europe_ratio: {market_pivot['usca_vs_europe_ratio'].mean():.3f} ± {market_pivot['usca_vs_europe_ratio'].std():.3f}")
print(f"- europe_market_share: {market_pivot['europe_market_share'].mean():.3f} ± {market_pivot['europe_market_share'].std():.3f}")
print(f"- latam_market_share: {market_pivot['latam_market_share'].mean():.3f} ± {market_pivot['latam_market_share'].std():.3f}")
print(f"- usca_market_share: {market_pivot['usca_market_share'].mean():.3f} ± {market_pivot['usca_market_share'].std():.3f}")

# Hiển thị vài ví dụ
print(f"\n🔍 Ví dụ market relationships:")
cols_to_show = ['order_date_only', 'europe_vs_latam_ratio', 'usca_vs_europe_ratio', 
                'europe_market_share', 'latam_market_share', 'usca_market_share']
print(market_pivot[cols_to_show].head())

🔄 Tính cross-market ratios...
✅ Market relationship features:
- europe_vs_latam_ratio: 0.998 ± 0.110
- usca_vs_europe_ratio: 1.004 ± 0.125
- europe_market_share: 0.333 ± 0.022
- latam_market_share: 0.335 ± 0.022
- usca_market_share: 0.332 ± 0.024

🔍 Ví dụ market relationships:
  order_date_only  europe_vs_latam_ratio  usca_vs_europe_ratio  \
0      2017-05-22               1.109756              1.005494   
1      2017-05-23               1.135802              0.739130   
2      2017-05-24               1.064327              0.895604   
3      2017-05-25               0.993789              1.081250   
4      2017-05-26               0.921053              0.954286   

   europe_market_share  latam_market_share  usca_market_share  
0             0.344045            0.310019           0.345936  
1             0.381743            0.336100           0.282158  
2             0.352713            0.331395           0.315891  
3             0.323887            0.325911           0.350202  
4    

## 7. 📈 Market Volatility and Trend Features
**Mục tiêu**: Tính 7-day trends, volatility và momentum cho từng market

In [9]:
print("📈 Tính Market Volatility và Trend Features...")

# Sort theo date để rolling window đúng
market_pivot = market_pivot.sort_values('order_date_only').reset_index(drop=True)

def calculate_trend_slope(series):
    """Tính slope của trend 7 ngày"""
    if len(series) < 2:
        return 0
    x = np.arange(len(series))
    slope = np.polyfit(x, series, 1)[0]
    return slope

# 1. 7-day rolling volatility (standard deviation)
window = 7
market_pivot['europe_7d_volatility'] = market_pivot['europe_orders'].rolling(window=window, min_periods=1).std()
market_pivot['latam_7d_volatility'] = market_pivot['latam_orders'].rolling(window=window, min_periods=1).std()
market_pivot['usca_7d_volatility'] = market_pivot['usca_orders'].rolling(window=window, min_periods=1).std()

# 2. 7-day rolling trend (slope)
market_pivot['europe_7d_trend'] = market_pivot['europe_orders'].rolling(window=window, min_periods=2).apply(calculate_trend_slope, raw=True)
market_pivot['latam_7d_trend'] = market_pivot['latam_orders'].rolling(window=window, min_periods=2).apply(calculate_trend_slope, raw=True)
market_pivot['usca_7d_trend'] = market_pivot['usca_orders'].rolling(window=window, min_periods=2).apply(calculate_trend_slope, raw=True)

# 3. Market volatility (tổng thể)
market_pivot['market_volatility'] = market_pivot['total_orders'].rolling(window=window, min_periods=1).std()

# 4. 7-day momentum (current vs 7-day average)
market_pivot['europe_7d_avg'] = market_pivot['europe_orders'].rolling(window=window, min_periods=1).mean()
market_pivot['europe_momentum'] = market_pivot['europe_orders'] / market_pivot['europe_7d_avg']

market_pivot['latam_7d_avg'] = market_pivot['latam_orders'].rolling(window=window, min_periods=1).mean()
market_pivot['latam_momentum'] = market_pivot['latam_orders'] / market_pivot['latam_7d_avg']

market_pivot['usca_7d_avg'] = market_pivot['usca_orders'].rolling(window=window, min_periods=1).mean()
market_pivot['usca_momentum'] = market_pivot['usca_orders'] / market_pivot['usca_7d_avg']

# Fill NaN values với 0 cho trends
trend_cols = ['europe_7d_trend', 'latam_7d_trend', 'usca_7d_trend']
market_pivot[trend_cols] = market_pivot[trend_cols].fillna(0)

print("✅ Market dynamics features:")
print(f"- 7d volatility: Europe={market_pivot['europe_7d_volatility'].mean():.2f}, LATAM={market_pivot['latam_7d_volatility'].mean():.2f}, USCA={market_pivot['usca_7d_volatility'].mean():.2f}")
print(f"- 7d trend: Europe={market_pivot['europe_7d_trend'].mean():.3f}, LATAM={market_pivot['latam_7d_trend'].mean():.3f}, USCA={market_pivot['usca_7d_trend'].mean():.3f}")
print(f"- Momentum: Europe={market_pivot['europe_momentum'].mean():.3f}, LATAM={market_pivot['latam_momentum'].mean():.3f}, USCA={market_pivot['usca_momentum'].mean():.3f}")
print(f"- Market volatility: {market_pivot['market_volatility'].mean():.2f}")

print(f"\n🔍 Ví dụ volatility và trends:")
trend_cols_show = ['order_date_only', 'europe_7d_volatility', 'europe_7d_trend', 'europe_momentum', 'market_volatility']
print(market_pivot[trend_cols_show].head(10))

📈 Tính Market Volatility và Trend Features...
✅ Market dynamics features:
- 7d volatility: Europe=12.83, LATAM=13.06, USCA=14.50
- 7d trend: Europe=-0.065, LATAM=-0.002, USCA=-0.220
- Momentum: Europe=0.999, LATAM=1.000, USCA=1.000
- Market volatility: 22.60

🔍 Ví dụ volatility và trends:
  order_date_only  europe_7d_volatility  europe_7d_trend  europe_momentum  \
0      2017-05-22                   NaN     0.000000e+00         1.000000   
1      2017-05-23              1.414214     2.000000e+00         1.005464   
2      2017-05-24              1.154701     2.875853e-14         0.996350   
3      2017-05-25             11.372481    -6.800000e+00         0.903955   
4      2017-05-26              9.889388    -3.800000e+00         0.990940   
5      2017-05-27             13.382825    -5.685714e+00         0.881159   
6      2017-05-28             12.733533    -4.571429e+00         0.952421   
7      2017-05-29             11.816050    -3.035714e+00         1.008425   
8      2017-05-30

In [10]:
# Merge market features vào dataset chính
print("🔗 Merge market features vào dataset chính...")

# Chọn các features để merge (không cần europe_orders, latam_orders, usca_orders vì đã có)
market_features_to_merge = [
    'order_date_only',
    'europe_vs_latam_ratio', 'usca_vs_europe_ratio',
    'europe_market_share', 'latam_market_share', 'usca_market_share',
    'total_orders', 'market_volatility',
    'europe_7d_volatility', 'latam_7d_volatility', 'usca_7d_volatility',
    'europe_7d_trend', 'latam_7d_trend', 'usca_7d_trend',
    'europe_momentum', 'latam_momentum', 'usca_momentum'
]

market_features_df = market_pivot[market_features_to_merge].copy()

# Merge vào df chính
df_before_merge = df.shape[0]
df = df.merge(market_features_df, on='order_date_only', how='left')
df_after_merge = df.shape[0]

print(f"✅ Merge thành công!")
print(f"- Trước merge: {df_before_merge} rows")
print(f"- Sau merge: {df_after_merge} rows")
print(f"- Số features mới: {len(market_features_to_merge) - 1}")  # -1 vì order_date_only không phải feature mới
print(f"- Tổng features hiện tại: {df.shape[1]}")

# Kiểm tra có missing values từ merge không
missing_after_merge = df[market_features_to_merge[1:]].isnull().sum().sum()
if missing_after_merge > 0:
    print(f"⚠️ Có {missing_after_merge} missing values sau merge")
else:
    print("✅ Không có missing values sau merge")

print(f"\n📊 Market features đã được thêm:")
for feature in market_features_to_merge[1:]:
    print(f"- {feature}: {df[feature].mean():.4f} ± {df[feature].std():.4f}")

🔗 Merge market features vào dataset chính...
✅ Merge thành công!
- Trước merge: 765 rows
- Sau merge: 765 rows
- Số features mới: 16
- Tổng features hiện tại: 51
⚠️ Có 12 missing values sau merge

📊 Market features đã được thêm:
- europe_vs_latam_ratio: 0.9984 ± 0.1103
- usca_vs_europe_ratio: 1.0043 ± 0.1250
- europe_market_share: 0.3328 ± 0.0223
- latam_market_share: 0.3353 ± 0.0216
- usca_market_share: 0.3318 ± 0.0238
- total_orders: 512.7765 ± 23.9232
- market_volatility: 22.6011 ± 6.7224
- europe_7d_volatility: 12.8297 ± 3.6047
- latam_7d_volatility: 13.0614 ± 3.8219
- usca_7d_volatility: 14.4958 ± 3.8777
- europe_7d_trend: -0.0654 ± 2.6830
- latam_7d_trend: -0.0018 ± 2.7993
- usca_7d_trend: -0.2203 ± 4.2847
- europe_momentum: 0.9994 ± 0.0728
- latam_momentum: 0.9995 ± 0.0752
- usca_momentum: 0.9996 ± 0.0821


## 8. ✅ Data Quality Validation
**Mục tiêu**: Kiểm tra missing values, outliers, distributions và correlations

In [11]:
print("✅ Data Quality Validation...")

# 1. Kiểm tra missing values chi tiết
print("❌ Missing values chi tiết:")
missing_counts = df.isnull().sum()
missing_counts = missing_counts[missing_counts > 0]
if len(missing_counts) > 0:
    print(missing_counts)
    print(f"\nTổng missing: {missing_counts.sum()}")
else:
    print("✅ Không có missing values!")

# 2. Xử lý missing values (forward fill cho time series)
print(f"\n🔧 Xử lý missing values...")
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(method='ffill').fillna(method='bfill')

# Kiểm tra lại
missing_after_fill = df.isnull().sum().sum()
print(f"✅ Missing values sau xử lý: {missing_after_fill}")

# 3. Kiểm tra outliers cho target đã normalize
print(f"\n📊 Kiểm tra outliers cho target normalized:")
target_col = 'order_count'
q1 = df[target_col].quantile(0.25)
q3 = df[target_col].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = df[(df[target_col] < lower_bound) | (df[target_col] > upper_bound)]
print(f"- Q1: {q1:.3f}, Q3: {q3:.3f}, IQR: {iqr:.3f}")
print(f"- Bounds: [{lower_bound:.3f}, {upper_bound:.3f}]")
print(f"- Outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")

# 4. Tổng kết features mới
print(f"\n📈 Tổng kết features đã thêm:")

new_features = [
    # Target normalization
    'order_count_original',
    # Cyclical features
    'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos', 
    'day_of_month_sin', 'day_of_month_cos',
    # Business calendar
    'is_month_end', 'is_quarter_end', 'week_of_year', 'week_of_year_sin', 'week_of_year_cos',
    # Market relationships
    'europe_vs_latam_ratio', 'usca_vs_europe_ratio',
    'europe_market_share', 'latam_market_share', 'usca_market_share', 'total_orders',
    # Market dynamics
    'market_volatility', 'europe_7d_volatility', 'latam_7d_volatility', 'usca_7d_volatility',
    'europe_7d_trend', 'latam_7d_trend', 'usca_7d_trend',
    'europe_momentum', 'latam_momentum', 'usca_momentum'
]

existing_new_features = [f for f in new_features if f in df.columns]
print(f"- Số features mới: {len(existing_new_features)}")
print(f"- Tổng features: {len(df.columns)}")

print(f"\n✅ Dataset optimized shape: {df.shape}")

✅ Data Quality Validation...
❌ Missing values chi tiết:
market_volatility       3
europe_7d_volatility    3
latam_7d_volatility     3
usca_7d_volatility      3
dtype: int64

Tổng missing: 12

🔧 Xử lý missing values...
✅ Missing values sau xử lý: 0

📊 Kiểm tra outliers cho target normalized:
- Q1: -0.635, Q3: 0.717, IQR: 1.353
- Bounds: [-2.665, 2.746]
- Outliers: 2 (0.3%)

📈 Tổng kết features đã thêm:
- Số features mới: 28
- Tổng features: 51

✅ Dataset optimized shape: (765, 51)


## 9. 💾 Export Optimized Dataset
**Mục tiêu**: Lưu dataset tối ưu và fitted scalers cho model training

In [12]:
# Export optimized dataset
output_path = '../dataset/supply_chain_optimized.csv'
scalers_dir = '../scalers/'

print("💾 Export optimized dataset...")

# Tạo thư mục scalers nếu chưa có
os.makedirs(scalers_dir, exist_ok=True)
os.makedirs('../dataset/', exist_ok=True)

# 1. Lưu dataset
df.to_csv(output_path, index=False)
print(f"✅ Đã lưu optimized dataset: {output_path}")
print(f"📊 Shape: {df.shape}")

# 2. Lưu target scaler (quan trọng cho model inference)
scaler_path = os.path.join(scalers_dir, 'target_scaler.pkl')
joblib.dump(target_scaler, scaler_path)
print(f"✅ Đã lưu target scaler: {scaler_path}")

# 3. Lưu metadata về features mới
metadata = {
    'original_features': 23,
    'optimized_features': df.shape[1],
    'new_features_added': df.shape[1] - 23,
    'target_normalization': 'StandardScaler applied to order_count',
    'cyclical_features': [
        'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos', 
        'day_of_month_sin', 'day_of_month_cos', 'week_of_year_sin', 'week_of_year_cos'
    ],
    'business_calendar': ['is_month_end', 'is_quarter_end', 'week_of_year'],
    'market_relationships': [
        'europe_vs_latam_ratio', 'usca_vs_europe_ratio',
        'europe_market_share', 'latam_market_share', 'usca_market_share'
    ],
    'market_dynamics': [
        'market_volatility', 'europe_7d_volatility', 'latam_7d_volatility', 'usca_7d_volatility',
        'europe_7d_trend', 'latam_7d_trend', 'usca_7d_trend',
        'europe_momentum', 'latam_momentum', 'usca_momentum'
    ],
    'processing_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'expected_improvement': 'MSE: 20,437 → 10,000-12,000 (50-60%)'
}

metadata_path = os.path.join(scalers_dir, 'optimization_metadata.pkl')
joblib.dump(metadata, metadata_path)
print(f"✅ Đã lưu metadata: {metadata_path}")

# 4. Summary report
print(f"\n🎯 SUMMARY - Giai đoạn 1 hoàn thành!")
print(f"=" * 50)
print(f"📊 Dataset original: 765 rows × 23 features")
print(f"📈 Dataset optimized: 765 rows × {df.shape[1]} features")
print(f"🆕 Features added: {df.shape[1] - 23}")
print(f"")
print(f"✅ Optimizations applied:")
print(f"   1. Target normalization (StandardScaler)")
print(f"   2. Cyclical encoding (8 features)")  
print(f"   3. Business calendar (3 features)")
print(f"   4. Market relationships (5 features)")
print(f"   5. Market dynamics (12 features)")
print(f"")
print(f"📁 Files saved:")
print(f"   - {output_path}")
print(f"   - {scaler_path}")
print(f"   - {metadata_path}")
print(f"")
print(f"🎯 Expected improvement: MSE 20,437 → 10,000-12,000 (50-60%)")
print(f"🚀 Ready for model training!")

print(f"\n📝 Để sử dụng trong training:")
print(f"1. Load data: pd.read_csv('{output_path}')")
print(f"2. Load scaler: joblib.load('{scaler_path}')")  
print(f"3. Update data_factory.py để sử dụng supply_chain_optimized.csv")
print(f"4. Implement weighted loss function trong training loop")

💾 Export optimized dataset...
✅ Đã lưu optimized dataset: ../dataset/supply_chain_optimized.csv
📊 Shape: (765, 51)
✅ Đã lưu target scaler: ../scalers/target_scaler.pkl
✅ Đã lưu metadata: ../scalers/optimization_metadata.pkl

🎯 SUMMARY - Giai đoạn 1 hoàn thành!
📊 Dataset original: 765 rows × 23 features
📈 Dataset optimized: 765 rows × 51 features
🆕 Features added: 28

✅ Optimizations applied:
   1. Target normalization (StandardScaler)
   2. Cyclical encoding (8 features)
   3. Business calendar (3 features)
   4. Market relationships (5 features)
   5. Market dynamics (12 features)

📁 Files saved:
   - ../dataset/supply_chain_optimized.csv
   - ../scalers/target_scaler.pkl
   - ../scalers/optimization_metadata.pkl

🎯 Expected improvement: MSE 20,437 → 10,000-12,000 (50-60%)
🚀 Ready for model training!

📝 Để sử dụng trong training:
1. Load data: pd.read_csv('../dataset/supply_chain_optimized.csv')
2. Load scaler: joblib.load('../scalers/target_scaler.pkl')
3. Update data_factory.py để s